In [18]:
import tensorflow.keras as tk
from facenet_pytorch import InceptionResnetV1, extract_face
from mtcnn.mtcnn import MTCNN
from keras_vggface.utils import preprocess_input, decode_predictions
from keras_vggface.vggface import VGGFace
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
from scipy.spatial.distance import cosine
from tqdm.notebook import tqdm
import cv2
from numpy import asarray

%matplotlib inline

In [2]:
#Initialize MTCNN face detector
mtcnn = MTCNN()

In [3]:
# Reading an image
image = plt.imread('images/Kenaj.jpg')

In [4]:
# Load image frame from video
vidcap = cv2.VideoCapture('train_sample_vids/aaqaifqrwn.mp4')
v_len = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))

In [5]:
# Detect Faces
faces = mtcnn.detect_faces(image)
for face in faces:
    print(face)

{'box': [481, 200, 271, 360], 'confidence': 0.9999642372131348, 'keypoints': {'left_eye': (551, 331), 'right_eye': (681, 336), 'nose': (615, 396), 'mouth_left': (552, 468), 'mouth_right': (674, 472)}}


In [4]:
# Loop through a video
batch_size = 32
frames = []
boxes = []
landmarks = []
view_frames = []
view_landmarks = []
view_boxes = []
for i in tqdm(range(v_len)):

    success, frame = vidcap.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = Image.fromarray(frame)
    frame = frame.resize([int(f * 0.25) for f in frame.size])
    frames.append(frame)

    if len(frames) >= batch_size:
        batch_boxes, i, batch_landmarks = mtcnn.detect(frames, landmarks = True)
        boxes.extend(batch_boxes)
        landmarks.extend(batch_landmarks)

        view_frames.append(frames[-1])
        view_boxes.append(boxes[-1])
        view_landmarks.append(landmarks[-1])

        frames = []

  0%|          | 0/300 [00:00<?, ?it/s]

C:\Users\Naj\anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
C:\Users\Naj\anaconda3\lib\site-packages\facenet_pytorch\models\utils\detect_face.py:183: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  batch_boxes, batch_points = np.array(batch_boxes), np.array(batch_points)
C:\Users\Naj\anaconda3\lib\site-packages\facenet_pytorch\models\mtcnn.py:339: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a li

In [6]:
# Draw boxes around detected faces
def highlight_faces(image_path, faces):

    img = plt.imread(image_path)
    plt.imshow(img)

    axes = plt.gca()

    for face in faces:
        x, y, width, height = face['box']
        face_border = Rectangle((x, y), width, height, fill = False, color = 'cyan')
        axes.add_patch(face_border)
    plt.show()


In [7]:
# Use this method to extract faces. Use when you know multiple faces are in an image.
def extract_faces(image_path, required_size = (224, 224)):
    img = plt.imread(image_path)
    mtcnn = MTCNN()
    faces = mtcnn.detect_faces(img)
    face_images = []

    for face in faces:
        x1, y1, width, height = face['box']
        x2, y2 = x1 + width, y1 + height

        face_boundary = image[y1:y2, x1:x2]

        face_image = Image.fromarray(face_boundary)
        face_image = face_image.resize(required_size)
        face_array = asarray(face_image)
        face_images.append(face_array)


    return face_images

In [27]:
face = mtcnn.detect_faces(plt.imread('images/Kenaj.jpg'))

{'box': [481, 200, 271, 360],
 'confidence': 0.9999642372131348,
 'keypoints': {'left_eye': (551, 331),
  'right_eye': (681, 336),
  'nose': (615, 396),
  'mouth_left': (552, 468),
  'mouth_right': (674, 472)}}

In [28]:
# Use to detect and extract a single face. Only use if one face is in the picture for accurate extraction results.
def extract_face(image_path, required_size = (224, 224)):
    img = plt.imread(image_path)
    mtcnn = MTCNN()
    face = mtcnn.detect_faces(img)

    x1, y1, width, height = face[0]['box']
    x2, y2 = x1 + width, y1 + height

    face_boundary = image[y1:y2, x1:x2]

    face_image = Image.fromarray(face_boundary)
    face_image = face_image.resize(required_size)
    face_array = asarray(face_image)
    return face_array

In [29]:
extracted_face = extract_face('images/Kenaj.jpg')

In [31]:
extracted_face = extract_face('images/Kenaj.jpg')
pixels = extracted_face.astype('float32')
samples = np.expand_dims(pixels, axis = 0)

samples = preprocess_input(samples, version = 2)

model = VGGFace(model = 'resnet50')

predictions = model.predict(samples)

results = decode_predictions(predictions)

1359872/1346516 [==============================] - 0s 0us/step


In [ ]:
results.

In [34]:
resnet50_features = VGGFace(model = 'resnet50',
                    include_top = False,
                    input_shape = (224, 224, 3),
                    pooling = 'avg')
features = resnet50_features.predict(samples)

In [ ]:
precompute_features = []
for i, folder in enumerate(folders):

In [10]:
def get_model_scores(faces):
    samples = asarray(faces, 'float32')
    # Prepare the data for model
    samples = preprocess_input(samples, version = 2)

    # Initializing VGGFACE model
    model = VGGFace(model = 'resnet50',
                    include_top = False,
                    input_shape = (224, 224, 3),
                    pooling = 'avg')
    # Prediction
    return model.predict(samples)

In [11]:
faces = [extract_faces(image_path)[0]
         for image_path in ['images/Kenaj.jpg', 'images/Naj2.jpg']]

model_scores = get_model_scores(faces)

In [33]:
model_scores

array([[1.20661765e-01, 1.60902596e+01, 6.11604977e+00, ...,
        5.09694242e+00, 0.00000000e+00, 0.00000000e+00],
       [4.45848167e-01, 1.16878948e+01, 8.65180874e+00, ...,
        4.71990919e+00, 0.00000000e+00, 1.57156773e-03]], dtype=float32)

In [34]:
if cosine(model_scores[0], model_scores[1]) <= 0.4:
    print('Faces Matched')

Faces Matched


In [12]:
def img_to_encoding(image_path, model):
    img = tk.preprocessing.image.load_img(image_path, target_size=(224, 224))
    img = np.around(np.array(img) / 255.0, decimals=12)
    x_train = np.expand_dims(img, axis=0)
    embedding = model.predict_on_batch(x_train)
    return embedding / np.linalg.norm(embedding, ord=2)

In [13]:
def get_embeddings(files):
    faces = [extract_faces(file) for file in files]
    samples = asarray(faces, 'float32')
    samples = preprocess_input(samples, version = 2)
    model = VGGFace(model = 'resnet50', include_top = False, input_shape = (224, 224, 3), pooling = 'avg')
    prediction = model.predict(samples)
    return prediction

In [14]:
def is_match(known_embedding, new_embedding, threshold = 0.5):
    score = cosine(known_embedding, new_embedding)
    if score <= threshold:
        print(f'Face is a match ({score} <= {threshold})')
    else:
        print(f'Face is not a match ({score} <= {threshold})')

In [15]:
files = ['images/danielle.png',
         'images/younes.jpg',
         'images/tian.jpg',
         'images/andrew.jpg',
         'images/kian.jpg',
         'images/dan.jpg',
         'images/sebastiano.jpg',
         'images/bertrand.jpg',
         'images/kevin.jpg',
         'images/felix.jpg',
         'images/benoit.jpg',
         'images/arnaud.jpg',
         'images/Kenaj.jpg',
         'images/Naj.jpg',
         'images/Naj2.jpg']
embeddings = get_embeddings(files)

ValueError: in user code:

    c:\users\naj\pycharmprojects\udemy_files_project\venv\lib\site-packages\keras\engine\training.py:1586 predict_function  *
        return step_function(self, iterator)
    c:\users\naj\pycharmprojects\udemy_files_project\venv\lib\site-packages\keras\engine\training.py:1576 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\users\naj\pycharmprojects\udemy_files_project\venv\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\users\naj\pycharmprojects\udemy_files_project\venv\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\users\naj\pycharmprojects\udemy_files_project\venv\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwa

In [ ]:
# Visualize Results

figure, axes = plt.subplots(3, 3, figsize = (18, 12))

for i in range(9):
    axes[int(i / 3), i % 3].imshow(view_frames[i])
    axes[int(i / 3), i % 3].axis('off')
    for box, landmark in zip(view_boxes[i], view_landmarks[i]):
        axes[int(i / 3), i % 3].scatter(*np.meshgrid(box[[0, 2]], box[[1, 3]]), s = 8)
        axes[int(i / 3), i % 3].scatter(landmark[:, 0], landmark[:, 1], s = 6)

img = Image.open('images/Naj2.jpg').resize((160, 160))
img_cropped = mtcnn(img)

In [10]:
img_probs = mtcnn(img_cropped.unsqueeze(0))
